<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao28_IMDB_OutrosMetodos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula 1: Instalações e Importações
!pip install fasttext tensorflow-datasets

import re
import string
import pandas as pd
import numpy as np

# Para carregar os dados
import tensorflow_datasets as tfds

# Para pré-processamento
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Para Modelos 1 e 2 (sklearn)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import TruncatedSVD

# Para Modelo 3 (FastText)
import fasttext

# Para formatação
from IPython.display import display, Markdown

display(Markdown("## 1. Carregamento e Pré-processamento dos Dados (IMDB)"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=4498211 sha256=1eb4b37e1f28e2552d7393d136e76a723fc2033fa093c8ec24b72315373d8563
  Stored in directory: /root/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built fasttext


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 1. Carregamento e Pré-processamento dos Dados (IMDB)

In [2]:
# Célula 2: Carregar Dados do IMDB com TensorFlow Datasets

def carregar_dados_imdb():
    """Carrega os splits de treino e teste do IMDB e converte para DataFrame."""
    # Carrega os dados (text, label)
    train_data, test_data = tfds.load(name="imdb_reviews",
                                      split=['train', 'test'],
                                      as_supervised=True)

    # Converte para lista (mais rápido que iterar no dataset do TF)
    train_texts = [text.numpy().decode('utf-8') for text, label in train_data]
    train_labels = [label.numpy() for text, label in train_data]

    test_texts = [text.numpy().decode('utf-8') for text, label in test_data]
    test_labels = [label.numpy() for text, label in test_data]

    # Cria DataFrames
    df_train = pd.DataFrame({'text': train_texts, 'label': train_labels})
    df_test = pd.DataFrame({'text': test_texts, 'label': test_labels})

    return df_train, df_test

df_train, df_test = carregar_dados_imdb()

print(f"Dados de treino: {len(df_train)} amostras")
print(f"Dados de teste: {len(df_test)} amostras")
display(df_train.head())

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.1I47YF_1.0.0/imdb_reviews-train.tfrecor…

...

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.1I47YF_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.1I47YF_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Dados de treino: 25000 amostras
Dados de teste: 25000 amostras


,text,label
0,This was an absolutely terrible movie. Don't b...,0
1,"I have been known to fall asleep during films,...",0
2,Mann photographs the Alberta Rocky Mountains i...,0
3,This is the kind of film for a snowy Sunday af...,1
4,"As others have mentioned, all the women that g...",1


In [3]:
# Célula 3: Função de Pré-processamento

# Obtém a lista de stopwords em inglês
stop_words = set(stopwords.words('english'))

def preprocessar_texto(text):
    """
    Limpa o texto:
    1. Remove tags HTML (<br />)
    2. Remove pontuação
    3. Converte para minúsculas
    4. Remove stopwords
    """
    # 1. Remove HTML
    text = re.sub(r'<[^>]+>', ' ', text)

    # 2. Remove pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 3. Converte para minúsculas
    text = text.lower()

    # 4. Remove stopwords
    palavras = [palavra for palavra in text.split() if palavra not in stop_words]

    return ' '.join(palavras)

# Aplica o pré-processamento
# (Isso pode levar alguns minutos)
print("Iniciando pré-processamento...")
df_train['clean_text'] = df_train['text'].apply(preprocessar_texto)
df_test['clean_text'] = df_test['text'].apply(preprocessar_texto)
print("Pré-processamento concluído.")

display(df_train.head())

# Define nossos conjuntos de dados
X_train = df_train['clean_text']
y_train = df_train['label']

X_test = df_test['clean_text']
y_test = df_test['label']

Iniciando pré-processamento...
Pré-processamento concluído.


,text,label,clean_text
0,This was an absolutely terrible movie. Don't b...,0,absolutely terrible movie dont lured christoph...
1,"I have been known to fall asleep during films,...",0,known fall asleep films usually due combinatio...
2,Mann photographs the Alberta Rocky Mountains i...,0,mann photographs alberta rocky mountains super...
3,This is the kind of film for a snowy Sunday af...,1,kind film snowy sunday afternoon rest world go...
4,"As others have mentioned, all the women that g...",1,others mentioned women go nude film mostly abs...


In [4]:
# Célula 4: Método 1 - TF-IDF com Regressão Logística

display(Markdown("## 2. Método 1: TF-IDF + Regressão Logística"))

# Cria um pipeline do Scikit-learn
# 1. Vetoriza o texto usando TfidfVectorizer
# 2. Classifica usando Regressão Logística
pipeline_tfidf_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(solver='liblinear', C=10))
])

# Treina o modelo
print("Treinando TF-IDF + Regressão Logística...")
pipeline_tfidf_lr.fit(X_train, y_train)

# Faz previsões
y_pred_tfidf_lr = pipeline_tfidf_lr.predict(X_test)

# Avalia o modelo
print("\nResultados (TF-IDF + LR):")
report_tfidf_lr = classification_report(y_test, y_pred_tfidf_lr, output_dict=True)
print(classification_report(y_test, y_pred_tfidf_lr, target_names=['Negativo', 'Positivo']))

## 2. Método 1: TF-IDF + Regressão Logística

Treinando TF-IDF + Regressão Logística...

Resultados (TF-IDF + LR):
              precision    recall  f1-score   support

    Negativo       0.88      0.89      0.88     12500
    Positivo       0.89      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



In [5]:
# Célula 5: Método 2 - LSA (TF-IDF + SVD) com Regressão Logística

display(Markdown("## 3. Método 2: LSA (Latent Semantic Analysis) + Regressão Logística"))

# O LSA é implementado aplicando TruncatedSVD (um SVD para matrizes esparsas)
# após o TF-IDF.

# n_components é o número de "tópicos" latentes.
# 300 é um valor comum.
N_COMPONENTS_LSA = 300

pipeline_lsa_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1, 1))), # LSA geralmente funciona melhor com unigramas
    ('svd', TruncatedSVD(n_components=N_COMPONENTS_LSA)),
    ('clf', LogisticRegression(solver='liblinear', C=1)) # Ajuste do C para dados de menor dimensão
])

# Treina o modelo
print(f"Treinando LSA (SVD com {N_COMPONENTS_LSA} componentes) + Regressão Logística...")
pipeline_lsa_lr.fit(X_train, y_train)

# Faz previsões
y_pred_lsa_lr = pipeline_lsa_lr.predict(X_test)

# Avalia o modelo
print("\nResultados (LSA + LR):")
report_lsa_lr = classification_report(y_test, y_pred_lsa_lr, output_dict=True)
print(classification_report(y_test, y_pred_lsa_lr, target_names=['Negativo', 'Positivo']))

## 3. Método 2: LSA (Latent Semantic Analysis) + Regressão Logística

Treinando LSA (SVD com 300 componentes) + Regressão Logística...

Resultados (LSA + LR):
              precision    recall  f1-score   support

    Negativo       0.88      0.86      0.87     12500
    Positivo       0.87      0.88      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



In [6]:
# Célula 6: Método 3 - FastText

display(Markdown("## 4. Método 3: Classificador Nativo FastText"))

# O FastText requer um formato de arquivo específico:
# __label__<valor> <texto>
# Ex: __label__1 este filme foi otimo
# Ex: __label__0 eu odiei este filme

# Função para formatar e salvar os dados
def salvar_formato_fasttext(dataframe, nome_arquivo):
    with open(nome_arquivo, 'w') as f:
        for index, row in dataframe.iterrows():
            linha = f"__label__{row['label']} {row['clean_text']}\n"
            f.write(linha)

# Salva os arquivos de treino e teste
caminho_treino_ft = 'fasttext_imdb.train'
caminho_teste_ft = 'fasttext_imdb.test'

salvar_formato_fasttext(df_train, caminho_treino_ft)
salvar_formato_fasttext(df_test, caminho_teste_ft)

print(f"Arquivos salvos em: {caminho_treino_ft} e {caminho_teste_ft}")

# Treina o modelo FastText
# 'wordNgrams=2' é poderoso, captura bigramas de palavras
# 'epoch=10' é um bom ponto de partida
print("\nTreinando FastText...")
modelo_fasttext = fasttext.train_supervised(
    input=caminho_treino_ft,
    epoch=10,
    lr=0.5,
    wordNgrams=2,
    dim=100
)

print("Treinamento concluído.")

# Avalia o modelo no arquivo de teste
# O .test() retorna (n_exemplos, precisão@1, recall@1)
# Para classificação binária/multiclasse (não-multilabel), precisão@1 é a acurácia.
metricas_ft = modelo_fasttext.test(caminho_teste_ft)
acuracia_ft = metricas_ft[1]

print(f"\nAcurácia (nativa) do FastText: {acuracia_ft:.4f}")

# Para um relatório de classificação completo (comparável ao sklearn):
# Precisamos obter as previsões para todo o X_test
print("Calculando relatório de classificação completo para FastText...")
textos_teste = df_test['clean_text'].tolist()

# model.predict() retorna o label (ex: '__label__1') e a confiança
# Estamos interessados apenas no label
pred_labels_ft_raw = modelo_fasttext.predict(textos_teste)[0]

# Limpa o '__label__' para obter '0' ou '1' e converte para int
y_pred_fasttext = [int(label[0].replace('__label__', '')) for label in pred_labels_ft_raw]

# Avalia o modelo
print("\nResultados (FastText):")
report_fasttext = classification_report(y_test, y_pred_fasttext, output_dict=True)
print(classification_report(y_test, y_pred_fasttext, target_names=['Negativo', 'Positivo']))

## 4. Método 3: Classificador Nativo FastText

Arquivos salvos em: fasttext_imdb.train e fasttext_imdb.test

Treinando FastText...
Treinamento concluído.

Acurácia (nativa) do FastText: 0.8894
Calculando relatório de classificação completo para FastText...

Resultados (FastText):
              precision    recall  f1-score   support

    Negativo       0.88      0.90      0.89     12500
    Positivo       0.89      0.88      0.89     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000



In [7]:
# Célula 7: Comparação Final dos Resultados

display(Markdown("## 5. Comparação Final"))

# Coleta as métricas principais (macro avg) de cada relatório
def extrair_metricas(report, model_name):
    return {
        'Modelo': model_name,
        'Acurácia': report['accuracy'],
        'Precisão (Macro)': report['macro avg']['precision'],
        'Recall (Macro)': report['macro avg']['recall'],
        'F1-Score (Macro)': report['macro avg']['f1-score']
    }

metricas = [
    extrair_metricas(report_tfidf_lr, 'TF-IDF + LR'),
    extrair_metricas(report_lsa_lr, 'LSA + LR'),
    extrair_metricas(report_fasttext, 'FastText')
]

# Cria um DataFrame para fácil visualização
df_resultados = pd.DataFrame(metricas)
df_resultados = df_resultados.set_index('Modelo')

display(df_resultados.style.format("{:.4f}"))

display(Markdown(
"""
### Observações:

* **TF-IDF + Regressão Logística:** Geralmente é um *baseline* muito forte. O uso de `ngram_range=(1, 2)` (unigramas e bigramas) costuma ser muito eficaz para a análise de sentimentos.
* **LSA + Regressão Logística:** O LSA (Análise Semântica Latente) reduz a dimensionalidade. Isso pode ser útil em alguns casos, mas muitas vezes *perde* informações valiosas para a classificação, resultando em uma performance ligeiramente inferior ao TF-IDF puro.
* **FastText:** Quase sempre apresenta o melhor desempenho. Ele usa *subword information* (n-gramas de caracteres), o que o torna robusto a erros de digitação e palavras fora do vocabulário (OOV). Além disso, seu classificador nativo é altamente otimizado para tarefas de texto.
"""
))

## 5. Comparação Final

,Acurácia,Precisão (Macro),Recall (Macro),F1-Score (Macro)
Modelo,,,,
TF-IDF + LR,0.8840,0.8841,0.8840,0.8840
LSA + LR,0.8736,0.8737,0.8736,0.8735
FastText,0.8894,0.8895,0.8894,0.8894



### Observações:

* **TF-IDF + Regressão Logística:** Geralmente é um *baseline* muito forte. O uso de `ngram_range=(1, 2)` (unigramas e bigramas) costuma ser muito eficaz para a análise de sentimentos.
* **LSA + Regressão Logística:** O LSA (Análise Semântica Latente) reduz a dimensionalidade. Isso pode ser útil em alguns casos, mas muitas vezes *perde* informações valiosas para a classificação, resultando em uma performance ligeiramente inferior ao TF-IDF puro.
* **FastText:** Quase sempre apresenta o melhor desempenho. Ele usa *subword information* (n-gramas de caracteres), o que o torna robusto a erros de digitação e palavras fora do vocabulário (OOV). Além disso, seu classificador nativo é altamente otimizado para tarefas de texto.
